In [89]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import os
from sklearn import metrics

PATH_MELANOMA = '../DermMel/test/Melanoma'
PATH_NOTMELANOMA = '../DermMel/test/NotMelanoma'
PATH_GENERATE_DIC_MEL = '../DermMel/geracao_dicionario/melanomas'
PATH_GENERATE_DIC_NOT_MEL = '../DermMel/geracao_dicionario/nao_melanomas'


files_path_dic_melanoma = os.listdir(PATH_GENERATE_DIC_MEL) 
files_path_dic_melanoma = [PATH_GENERATE_DIC_MEL + '/' + path for path in files_path_dic_melanoma]

files_path_dic_nao_melanoma = os.listdir(PATH_GENERATE_DIC_NOT_MEL) 
files_path_dic_nao_melanoma = [PATH_GENERATE_DIC_NOT_MEL + '/' + path for path in files_path_dic_nao_melanoma]

dic_des = []
dic_nao_melanoma = []
palavras = []
palavras_nao_melanoma = []
array_desc = []

for path_img in files_path_dic_melanoma:
    
    img = cv2.imread(path_img,0) # Leitura da imagem com filtro em escada de cinza
    
    sift = cv2.xfeatures2d.SIFT_create()

    # Encontrando os pontos-de-interesse
    kp = sift.detect(img,None)

    # computando o vetor-de-característica para cada ponto detectado
    kp, des = sift.compute(img, kp)
                    
    dic_des.extend(des)       


# Clusterização por meio de Kmeans
K = 50

kmeans_model = KMeans(n_clusters=K).fit(dic_des) #dic_des são os vetores-de-características concatenados

# criando o dicionário de palavras
palavras = np.array(kmeans_model.cluster_centers_)

# print("Dicionario 1")
# print(len(palavras))

for path_img in files_path_dic_nao_melanoma:
    img = cv2.imread(path_img,0) # Leitura da imagem com filtro em escada de cinza
        
    sift = cv2.xfeatures2d.SIFT_create()
    
    kp = sift.detect(img,None)
    
    kp, des = sift.compute(img, kp)
    
    dic_nao_melanoma.extend(des)
    

kmeans_model = KMeans(n_clusters=K).fit(dic_nao_melanoma) # dic_nao_melanoma são os vetores-de-características concatenados

# criando o dicionário de palavras
palavras_nao_melanomas = np.array(kmeans_model.cluster_centers_)

# print("Palavras não melanomas")
# print(len(palavras_nao_melanomas))

array_desc.extend(palavras)
array_desc.extend(palavras_nao_melanomas)

# print("Concatenação")
# print(len(array_desc))

Dicionario 1
50
Palavras não melanomas
50
Concatenação
100


In [90]:
'''
Método de extração de arquivo .arff. Autor: Glauco Vitor.
'''

def gravar_arquivo_arff(base_teste, classes): # 0 para NAO_MELANOMA e 1 para MELANOMA
    tam = len(base_teste[0][0])
    file = open('dic2_50.arff','w') 
 
    file.write('@relation cancer\n') 
    for i in range(tam):
        file.write('@attribute '+ str(i) +' NUMERIC\n') 
    
    file.write('@attribute classes {')
    
    a = set(classes)
    
    for i in a:
        file.write(str(i)+',')
    
    file.write('}')    
    
    for i in range(tam):
         len(set(classes))
    
    file.write('\n@data\n') 

    for item in base_teste:
        for i in range(len(item[0])):
            file.write("%s," % str(item[0][i])) 
        file.write("%s\n" % item[1])    
 
    file.close() 
    print('arquivo gravado')


In [91]:
'''
Distância euclidiana para a atribuição dos centróides a cada elemento da imagem.
'''

def distancia(a, b):
    M = len(a)
    soma = 0
    for i in range(M):
        soma = soma + ((a[i]-b[i])**2)
    return np.sqrt(soma)


In [92]:
'''
Método de extração para todas as imagens da base de treinamento. 
O vetor de descrição tomado por base é o que foi gerado no início.
'''

def extrair_caracteristica(img):
    sift = cv2.xfeatures2d.SIFT_create()
    # encontrando os pontos-de-interesse
    kp = sift.detect(img,None)
#     print('Qtde Pontos Detectados: ', len(kp))
    
    kp, des = sift.compute(img, kp)
    pontos_rotulados = []

    for ponto in des:
        min = 9999
        label = -1
        for i in range(len(array_desc)): #Tem o dobro do tamanho agora.
            dist = distancia(ponto, array_desc[i])
            if(dist < min):
                min = dist
                label = i
        pontos_rotulados.append(label)
    
#     hist = np.zeros(K)
    hist = np.zeros(2*K) # Tenho 6 cluster para um K=3, pois são 2 dicionários.   
    
#     print("historico zeros")
#     print(hist)
    
#     print("Pontos rotulados")
#     print(pontos_rotulados)

    for i in pontos_rotulados:
        hist[i] += 1
    
#     print(hist)
    return hist # Frequência em que cada palavra centroide aparece

In [93]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn import metrics

files_path_nao_melanoma = os.listdir(PATH_NOTMELANOMA) 
files_path_nao_melanoma = [PATH_NOTMELANOMA+'/'+path for path in files_path_nao_melanoma]
files_path_melanoma = os.listdir(PATH_MELANOMA)
files_path_melanoma = [PATH_MELANOMA+'/'+path for path in files_path_melanoma]


all_image_paths = [files_path_nao_melanoma, files_path_melanoma]

base = []
classes = []

#extraindo as características das imagens na base de dados
for i in range(len(all_image_paths)):
    for img_path in all_image_paths[i][:500]:
        imagem = cv2.imread(img_path)
        caracteristica = extrair_caracteristica(imagem)
        classes.append(i)
        base.append((caracteristica, i))
    print('Classe final: '+str(i))

gravar_arquivo_arff(base, classes)

Classe final: 0
Classe final: 1
arquivo gravado
